In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
from tqdm import *
import pandas as pd

In [2]:
# chromedriver = '/Users/datakim/workspace/selenium_learning/chromedriver' 
driver = webdriver.Chrome() 

#리뷰가 많은 음식점 임의로 선정 
key_word = '아산병원푸드코트'
naver_map_search_url = f"https://m.map.naver.com/search2/search.naver?query="+key_word+"&sm=hty&style=v5"

driver.get(naver_map_search_url)
time.sleep(1)

#검색 후 첫 번째 검색결과의 주소 가져오기 - 주소로 하지 않고 클릭으로 넘어가게 해도 됐을 듯함
sid_num = driver.find_element(By.CLASS_NAME, '_item._lazyImgContainer').get_attribute('data-sid')
# element.click()
# # df.iloc[i,-1] = driver.find_element(By.CLASS_NAME,"li._item _lazyImgContainer").get_attribute('data-sid')

dscrp_url = "https://m.place.naver.com/restaurant/"+sid_num+"/review"
driver.get(dscrp_url)
time.sleep(1)

#-------------------
#사용자 리뷰 창 url과 사용자 이름을 모을 리스트 선언
userLink = []
userID = []

users = driver.find_elements(By.CLASS_NAME, "Hazns")
#print(users)
for user in users:
#     driver.find_elements(By.XPATH,"//*[@id="app-root]/div/div/div/div[7]/div[2]/div[3]/div[1]/ul/li[1]/div[1]")
    #userLink.append(driver.find_element(By.CSS_SELECTOR,"a:nth-child(1)").get_attribute('href'))
    #userLink.append(driver.find_element(By.CLASS_NAME, "Lia3P").get_attribute('href'))
    # 그냥 링크로 수집하니 사진모음 창이 기본창이라 텍스트 리뷰 창 주소에 고유번호를 입력하도록 함
    link_num = user.get_attribute('href').split('/')[4]
    userLink.append('https://m.place.naver.com/my/'+link_num+'/review?categoryId=&filter=all&sort=recentDesc')
    userID.append(user.find_element(By.CLASS_NAME, "sBWyy").text)
# print(userID)


In [3]:
#데이터프레임에 저장
user_url_df = pd.DataFrame({'userID' : userID, 'naver_map_url' : userLink})
user_url_df

,userID,naver_map_url
0,쩡3511,https://m.place.naver.com/my/5ccc0f018f87a842b...
1,정20,https://m.place.naver.com/my/5c19030523e21766a...
2,초긍미래,https://m.place.naver.com/my/5c8d8bb8fa12ad207...
3,희3974,https://m.place.naver.com/my/5d1f83e22efe54095...
4,Nado79,https://m.place.naver.com/my/5f1459289ec8258e4...
5,mk542,https://m.place.naver.com/my/5c6e6ada4bb61f68d...
6,토마토0000,https://m.place.naver.com/my/5c9254e99fc1e1b99...
7,베리베리별99,https://m.place.naver.com/my/5efec7fca7f1cdd89...
8,토미n제리,https://m.place.naver.com/my/5dbdb0908f87a842b...
9,달봉이애미,https://m.place.naver.com/my/5f3caf50a602e98f5...


In [4]:
user_url_df.to_csv("user_url_df")

In [5]:
#스크롤 내리는 함수
def repeat_scroll(driver):
    
    #스크롤 내리기 전 위치
    scroll_location = driver.execute_script("return document.body.scrollHeight")

    while True:
        #현재 스크롤의 가장 아래로 내림
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

        #전체 스크롤이 늘어날 때까지 대기
        time.sleep(2)

        #늘어난 스크롤 높이
        scroll_height = driver.execute_script("return document.body.scrollHeight")

        #늘어난 스크롤 위치와 이동 전 위치 같으면(더 이상 스크롤이 늘어나지 않으면) 종료
        if scroll_location == scroll_height:
            break

        #같지 않으면 스크롤 위치 값을 수정하여 같아질 때까지 반복
        scroll_location = driver.execute_script("return document.body.scrollHeight")

In [6]:
#사용자별 음식점 방문기록 가져오기
# 각 데이터들을 미리 리스트에 담은 다음, 마지막에 데이터 프레임에 합치려고 함
blog_review_cate = []
restaur_names = []
review_text = []
cate_column = []
user_name = []
user_num = 0

driver = webdriver.Chrome() 

#사용자 리뷰 url마다 돌면서 사용자의 음식점 방문기록을 수집
for url in user_url_df['naver_map_url']:

    driver.get(url)
    time.sleep(2)
        
        
    #카테고리별로 클릭해서 음식점과 음식점 리뷰 수집하기
    #전체 카테고리는 빼고 수집
    cate_buttons = driver.find_elements(By.CLASS_NAME, "_2Rz9w.dWrkK")[1:]

    for cate_button in cate_buttons:
        time.sleep(1)
        cate_button.click()
        time.sleep(1)

        #버튼의 이름(분류명) 저장
        cate_name = cate_button.find_element(By.CLASS_NAME, "EDjK9").text
        #스크롤 내려서 안보이는 음식점까지 보이게 하기 
        repeat_scroll(driver)
        
        #다시 스크롤 올리기
        driver.execute_script("window.scrollTo(0,0)")

        restaurants = driver.find_elements(By.CLASS_NAME, "_2teBZ")
        restaurant_names = driver.find_elements(By.CLASS_NAME, "_2v2rP")
        

        #음식점마다 리뷰텍스트 수집
        #리뷰텍스트가 없으면 null로 입력 
        for restaurant in restaurants:
            try:
                review_text.append(restaurant.find_element(By.CLASS_NAME, "_1ZYIc").text)
            except Exception as noreview:
                review_text.append('null')
                
        #음식점명과 카테고리명을 리스트로 수집
        for restaurant_name in restaurant_names:
            restaur_names.append(restaurant_name.text)
            cate_column.append(cate_name)
            user_name.append(user_url_df['userID'].iloc[user_num])
    
    user_num +=1
    
driver.quit()

In [7]:
print(len(restaur_names))
print(len(review_text))
print(len(cate_column))
print(len(user_name))

3681
3681
3681
3681


잘 가져왔음

In [10]:
#데이터프레임 만들기
review_df = pd.DataFrame(
                {'userID' : user_name,
                 'restaur_names' : restaur_names,
                 'review_text' : review_text,
                 'cate_column' : cate_column},
                )
review_df

,userID,restaur_names,review_text,cate_column
0,쩡3511,달마이어,항상 배앓이를 해서 락토프리로 주문하는데요. 연유라떼 주문해서 차타고 집에 왔는데 ...,카페
1,쩡3511,큰숲빵집 부곡점,오늘도 소금빵 맛있었어요!!,카페
2,쩡3511,오투하우스카페,편안하게 가성비 있는 커피 마시기에 좋아요~~~,카페
3,쩡3511,별의감성,바스크치즈케이크가 정말 맛있는 곳이에요! 커피 머신을 바꿨다고 하시는데 크레마덕에 ...,카페
4,쩡3511,배스킨라빈스 안산부곡점,요즘 spc 불매로 잘 안먹었는데 아이들이 마땅히 먹을 아이스크림 가게가 없어서 주...,카페
...,...,...,...,...
3676,달봉이애미,보그너정촌점,null,기타
3677,달봉이애미,라이프믹스,null,기타
3678,달봉이애미,록퍼드 아이웨어,null,기타
3679,달봉이애미,집꾸미기,null,기타


In [11]:
review_df.to_csv('raw_user_data ')